In [88]:
from datetime import datetime
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import csv
import torch

from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import datasets, transforms


# Re-Writing dataset

In [2]:
# Function used to count number of samples in each class in each set
def count_files_in_subfolders(parent_folder):
    subfolders = ['wet_asphalt_smooth', 'wet_concrete_smooth', 'wet_gravel']
    wet_asphalt_smooth = 0
    wet_concrete_smooth = 0
    wet_gravel = 0
    for subfolder in subfolders:
        path = os.path.join(parent_folder, subfolder)
        if os.path.exists(path):
            count = sum([len(files) for r, d, files in os.walk(path)])
            print(f"The {subfolder} folder contains {count} files.")
            
            if subfolder == 'wet_asphalt_smooth':
                wet_asphalt_smooth += count
            if subfolder == 'wet_concrete_smooth':
                wet_concrete_smooth += count
            if subfolder == 'wet_gravel':
                wet_gravel += count
        else:
            print(f"The {subfolder} folder does not exist in the specified path.")
    return [wet_asphalt_smooth, wet_concrete_smooth, wet_gravel]

# Prints out the number of files in each folder and accumulates them
print('Train')
train_total = count_files_in_subfolders("Group_9_wet_smooth/Train")

print('Test')
test_total = count_files_in_subfolders("Group_9_wet_smooth/Test")

print('Validation')
valid_total = count_files_in_subfolders("Group_9_wet_smooth/Valid")

grand_total = np.array(train_total) + np.array(test_total) + np.array(valid_total)

print(f'Total wet_asphalt_smooth: {grand_total[0]}')
print(f'Total wet_concrete_smooth: {grand_total[1]}')
print(f'Total wet_gravel: {grand_total[2]}')

# Calculate the ideal based on the minority class
ideal_train = round(np.min(grand_total) * 0.8,0)
ideal_test = round(np.min(grand_total) * 0.1,0)
ideal_valid = round(np.min(grand_total) * 0.1,0)

print(f"\nIdeal split for 80% train, 10% test, 10% validation:")
print(f"Train: {ideal_train} files")
print(f"Test: {ideal_test} files")
print(f"Validation: {ideal_valid} files")


Train
The wet_asphalt_smooth folder contains 79404 files.
The wet_concrete_smooth folder contains 66955 files.
The wet_gravel folder contains 36515 files.
Test
The wet_asphalt_smooth folder contains 79 files.
The wet_concrete_smooth folder contains 160 files.
The wet_gravel folder contains 2351 files.
Validation
The wet_asphalt_smooth folder contains 821 files.
The wet_concrete_smooth folder contains 821 files.
The wet_gravel folder contains 821 files.
Total wet_asphalt_smooth: 80304
Total wet_concrete_smooth: 67936
Total wet_gravel: 39687

Ideal split for 80% train, 10% test, 10% validation:
Train: 31750.0 files
Test: 3969.0 files
Validation: 3969.0 files


In [3]:
# Reads a file from old path and writes to new path
def read_and_write_img(old_path,new_path):
    try:
        img = cv2.imread(old_path)
        img = cv2.resize(img, (96, 64), interpolation=cv2.INTER_CUBIC)
        cv2.imwrite(new_path, img)
    except:
        print(f'failed on {old_path}')

# Takes in old and new dataset folders, the names of the classes and the number of files to be written to the test and validation sets
def balance_dataset(old_dataset_folder, new_dataset_folder, classes, test_valid_count):
    sets = ['Train', 'Test', 'Valid']
    file_lists = {}

    # Load and shuffle file lists, shuffling done to ensure excess files taken from old training sets are random.
    for set_name in sets:
        for class_name in classes:
            old_path = os.path.join(old_dataset_folder, set_name, class_name)
            files = os.listdir(old_path)
            random.shuffle(files)
            file_lists[(set_name, class_name)] = files

    # Process Validation and Test sets
    for set_name in ['Valid', 'Test']:
        print(set_name) # For debugging purposes
        for class_name in classes:
            print(class_name)
            new_path = os.path.join(new_dataset_folder, set_name, class_name)
            os.makedirs(new_path, exist_ok=True)

            # While there are not enough files in the new test or valid folder for a given class
            # This will try take a file from the old test/valid folder and if it has run out it will 
            # instead take the extra it needs from the training class
            while len(os.listdir(new_path)) < test_valid_count:
                if len(file_lists[(set_name, class_name)]) > 0:
                    file_name = file_lists[(set_name, class_name)].pop(0)
                    old_img_path = os.path.join(old_dataset_folder, set_name, class_name, file_name)
                    new_img_path = os.path.join(new_path, file_name)

                    read_and_write_img(old_img_path,new_img_path)
                elif len(file_lists[('Train', class_name)]) > 0:
                    file_name = file_lists[('Train', class_name)].pop(0)
                    old_img_path = os.path.join(old_dataset_folder, 'Train', class_name, file_name)
                    new_img_path = os.path.join(new_path, file_name)

                    read_and_write_img(old_img_path,new_img_path)
                else:
                    break

    print('Train')
    # Process Train set
    for class_name in classes:
        print(class_name)
        new_path = os.path.join(new_dataset_folder, 'Train', class_name)
        os.makedirs(new_path, exist_ok=True)

        while file_lists[('Train', class_name)]:
            file_name = file_lists[('Train', class_name)].pop(0)
            old_img_path = os.path.join(old_dataset_folder, 'Train', class_name, file_name)
            new_img_path = os.path.join(new_path, file_name)
            read_and_write_img(old_img_path,new_img_path)

old_dataset_folder = "Group_9_wet_smooth"
new_dataset_folder = "New_dataset"
classes = ['wet_asphalt_smooth', 'wet_concrete_smooth', 'wet_gravel']
test_valid_count = 3969

# Commented out as this has already been completed.
# balance_dataset(old_dataset_folder, new_dataset_folder, classes, test_valid_count)


# Data Loading 

In [97]:
# Creates a dataset given img directories
class ImageDataset(Dataset):
    def __init__(self, img_dirs, transform=None):
        self.img_dirs = img_dirs
        self.transform = transform
        self.images = []
        self.labels = []
        # Utilises the enumerate as the label
        for i, img_dir in enumerate(img_dirs):
            for img_name in os.listdir(img_dir):
                img_path = os.path.join(img_dir, img_name)
                self.images.append(img_path)
                self.labels.append(i)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = cv2.imread(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

# Creates train, validation and test loaders
def create_data_loaders(train_dirs, valid_dirs, test_dirs, batch_size, train_samples):
    # Conversion to tensor also normalises images between 0 and 1
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # Creates datasets
    train_datasets = ImageDataset(train_dirs, transform=transform)
    valid_datasets = ImageDataset(valid_dirs, transform=transform)
    test_datasets = ImageDataset(test_dirs, transform=transform)


    

    # Given the massive imbalance in the samples in each class a WeightedRandomSampler is used to allow the model to learn properly
    # Weights len is of len(num_samples) to give each a weight.
    weights_len = len(os.listdir(train_dirs[0])) + len(os.listdir(train_dirs[1])) + len(os.listdir(train_dirs[2]))
    sampler_weights = np.zeros(weights_len)

    # Pytorch takes all the values in a weighted random sampler and normalises them between 0 and 1 so the sampler_weights array does not have to sum to 1
    # All of the samples for each class are given a weight of 1 / Class_num_samples which leads to roughly equal numbers of each class being returned each epoch
    sampler_weights[:len(os.listdir(train_dirs[0]))] = 1/len(os.listdir(train_dirs[0]))
    sampler_weights[len(os.listdir(train_dirs[0])):-len(os.listdir(train_dirs[2]))] = 1/len(os.listdir(train_dirs[1]))
    sampler_weights[-len(os.listdir(train_dirs[2])):] = 1/len(os.listdir(train_dirs[2]))

    # sampler is created with the weights, a configurable number of samples per epoch and replacement enabled.
    sampler = WeightedRandomSampler(weights=sampler_weights, num_samples=train_samples, replacement=True)
    
    # Loaders are created from datasets with configurable batch_size
    train_loader = DataLoader(train_datasets, batch_size=batch_size, sampler=sampler)
    valid_loader = DataLoader(valid_datasets, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_datasets, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader

# Model Training

In [ ]:
def train_model(model, train_loader, valid_loader, test_loader, epochs, optimizer, criterion, title, train_samples, patience=5):
    
    # All of the variables needed to train a model are passed to the train model function to allow it to be reused.

    # GPU utilised for training.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Timestamp taken for saving of models and stats of each training run
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    train_loss_list, train_acc_list , valid_loss_list, valid_acc_list , test_loss_list , test_acc_list  = [],[],[],[],[],[]     
    best_valid_loss = float('inf')
    patience_counter = 0

    # Makes folder for logging
    os.makedirs(f'models/{timestamp}', exist_ok=True)
    # Opens csv to log Loss and Accuracy
    with open(f'models/{timestamp}/training_stats.csv', mode='w', newline='') as csvfile:
        fieldnames = ['Epoch', 'Train Loss', 'Train Acc', 'Valid Loss', 'Valid Acc', 'Test Loss', 'Test Acc']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for epoch in range(epochs):
            train_loss = 0.0
            train_correct = 0
            model.train()
            # Runs through all values in trainloader learns and accumulates loss and accuracy
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                labels = labels.to(torch.long)  
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                train_correct += (predicted == labels).sum().item()

            # Accumulates loss and accuracy of validation set
            valid_loss = 0.0
            valid_correct = 0
            model.eval()
            with torch.no_grad():
                for images, labels in valid_loader:
                    images, labels = images.to(device), labels.to(device)
                    labels = labels.to(torch.long)  
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    valid_loss += loss.item() * images.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    valid_correct += (predicted == labels).sum().item()

            # Accumulates loss and accuracy of test set
            test_loss = 0.0
            test_correct = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    labels = labels.to(torch.long)  
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    
                    test_loss += loss.item() * images.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    test_correct += (predicted == labels).sum().item()

            # Gets average loss and average accuracy for train, validation and test
            # Train_samples = configurable length of the trainloader
            train_loss /= train_samples;                train_acc = train_correct / train_samples
            valid_loss /= len(valid_loader.dataset);    valid_acc = valid_correct / len(valid_loader.dataset)
            test_loss /= len(test_loader.dataset);      test_acc = test_correct / len(test_loader.dataset)

            train_loss_list.append(train_loss),train_acc_list.append(train_acc),valid_loss_list.append(valid_loss),
            valid_acc_list.append(valid_acc),test_loss_list.append(test_loss),test_acc_list.append(test_acc)

            print(f'Epoch: {epoch}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

            writer.writerow({'Epoch': epoch,
                             'Train Loss': train_loss,
                             'Train Acc': train_acc,
                             'Valid Loss': valid_loss,
                             'Valid Acc': valid_acc,
                             'Test Loss': test_loss,
                             'Test Acc': test_acc,})
            
            # Updates best loss achieved, resets patience counter and saves the model
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                patience_counter = 0
                torch.save(model.state_dict(), f'models/{timestamp}/{epoch}.chkpt')
            # else increments the patience counter and if it has reached a limit stops the training.
            # Patience counter is implemented so if the model is overfitting resources will not be wasted.
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print('Early stopping')
                    break
        
        # Plots the models loass and accuracy.
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.plot(train_loss_list, label='Train')
        plt.plot(valid_loss_list, label='Valid')
        plt.plot(test_loss_list, label='Test')
        plt.title(f'{title} - Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(train_acc_list, label='Train')
        plt.plot(valid_acc_list, label='Valid')
        plt.plot(test_acc_list, label='Test')
        plt.title(f'{title} - Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()


# Model definition

In [128]:
# Implementation of: 
# Woo, Sanghyun, Jongchan Park, Joon-Young Lee, and In So Kweon. 
# "Cbam: Convolutional block attention module." 
# In Proceedings of the European conference on computer vision (ECCV), pp. 3-19. 2018.
# implemented by Peachypie98 on github at the following url https://github.com/Peachypie98/CBAM
class CAM(nn.Module):
    def __init__(self, channels, r):
        super(CAM, self).__init__()
        self.channels = channels
        self.r = r
        self.linear = nn.Sequential(
            nn.Linear(in_features=self.channels, out_features=self.channels//self.r, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=self.channels//self.r, out_features=self.channels, bias=True))

    def forward(self, x):
        max = F.adaptive_max_pool2d(x, output_size=1)
        avg = F.adaptive_avg_pool2d(x, output_size=1)
        b, c, _, _ = x.size()
        linear_max = self.linear(max.view(b,c)).view(b, c, 1, 1)
        linear_avg = self.linear(avg.view(b,c)).view(b, c, 1, 1)
        output = linear_max + linear_avg
        output = F.sigmoid(output) * x
        return output

class SAM(nn.Module):
    def __init__(self, bias=False):
        super(SAM, self).__init__()
        self.bias = bias
        self.conv = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3, dilation=1, bias=self.bias)

    def forward(self, x):
        max = torch.max(x,1)[0].unsqueeze(1)
        avg = torch.mean(x,1).unsqueeze(1)
        concat = torch.cat((max,avg), dim=1)
        output = self.conv(concat)
        output = F.sigmoid(output) * x 
        return output 
    
class CBAM(nn.Module):
    def __init__(self, channels, r):
        super(CBAM, self).__init__()
        self.channels = channels
        self.r = r
        self.sam = SAM(bias=False)
        self.cam = CAM(channels=self.channels, r=self.r)

    def forward(self, x):
        output = self.cam(x)
        output = self.sam(output)
        return output + x

# RCNET architecture implemented from the paper:
# Dewangan, Deepak Kumar, and Satya Prakash Sahu. 
# "RCNet: road classification convolutional neural networks for intelligent vehicle system." 
# Intelligent Service Robotics 14, no. 2 (2021): 199-214.

# Added additional attention layers based on the CBAM architecture to improve performance
class RCNet_attention(nn.Module):
    def __init__(self):
        super(RCNet_attention, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.CBAM1 = CBAM(64,4)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout()
        self.upsample1 = nn.Upsample(scale_factor=2)
        self.conv7 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.conv8 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.CBAM2 = CBAM(128,4)
        self.bn1 = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout()
        self.fc1 = nn.Linear(12288, 256, bias=True)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 256, bias=True)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 3, bias=True)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = self.CBAM1(x)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool3(x)
        x = self.dropout1(x)
        x = self.upsample1(x)
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = self.CBAM2(x)
        x = self.bn1(x)
        x = self.pool4(x)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = self.bn2(x)
        x = F.relu(self.fc2(x))
        x = self.bn3(x)
        x = self.fc3(x)
        return x


# Running Code

In [140]:
train_dirs = ["New_dataset/Train/wet_asphalt_smooth", "New_dataset/Train/wet_concrete_smooth", "New_dataset/Train/wet_gravel"]
valid_dirs = ["New_dataset/Valid/wet_asphalt_smooth", "New_dataset/Valid/wet_concrete_smooth", "New_dataset/Valid/wet_gravel"]
test_dirs = ["New_dataset/Test/wet_asphalt_smooth", "New_dataset/Test/wet_concrete_smooth", "New_dataset/Test/wet_gravel"]
batch_size = 128
train_samples = 100000 # total number of samples in the train set
# train_samples = 20000
train_loader, valid_loader, test_loader = create_data_loaders(train_dirs, valid_dirs, test_dirs, batch_size, train_samples)

In [141]:
# model = RCNet()
model = RCNet_attention()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),0.00001,)
train_model(model,train_loader,valid_loader,test_loader,50,optimizer,criterion,'First Training run of RCNet no attention', train_samples, 50)

Epoch: 1/50, Train Loss: 0.6935, Train Acc: 0.6881, Valid Loss: 0.4803, Valid Acc: 0.8049, Test Loss: 0.4721, Test Acc: 0.8149
Epoch: 2/50, Train Loss: 0.4575, Train Acc: 0.8202, Valid Loss: 0.8056, Valid Acc: 0.6935, Test Loss: 0.8015, Test Acc: 0.6926
Epoch: 3/50, Train Loss: 0.4109, Train Acc: 0.8394, Valid Loss: 0.4112, Valid Acc: 0.8313, Test Loss: 0.3926, Test Acc: 0.8459
Epoch: 4/50, Train Loss: 0.3814, Train Acc: 0.8518, Valid Loss: 0.5109, Valid Acc: 0.8028, Test Loss: 0.4816, Test Acc: 0.8183
Epoch: 5/50, Train Loss: 0.3615, Train Acc: 0.8597, Valid Loss: 0.5353, Valid Acc: 0.7895, Test Loss: 0.5008, Test Acc: 0.8062
Epoch: 6/50, Train Loss: 0.3373, Train Acc: 0.8712, Valid Loss: 0.3579, Valid Acc: 0.8576, Test Loss: 0.3324, Test Acc: 0.8776
Epoch: 7/50, Train Loss: 0.3187, Train Acc: 0.8778, Valid Loss: 0.4466, Valid Acc: 0.8316, Test Loss: 0.3853, Test Acc: 0.8576
Epoch: 8/50, Train Loss: 0.3015, Train Acc: 0.8849, Valid Loss: 0.3588, Valid Acc: 0.8545, Test Loss: 0.3142, T

# Model evaluation

### Learning and accuracy curves

### Confusion matrices